In [1]:
!pip install groq python-dotenv numpy tqdm datasets


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
import time
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

c:\Users\pjhic\Downloads\7(1)-LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since gsm8k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'main' at C:\Users\pjhic\.cache\huggingface\datasets\gsm8k\main\0.0.0\cc7b047b6e5bb11b4f1af84efc572db110a51b3c (last modified on Fri Jan 30 14:05:07 2026).


In [3]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [4]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello world! It's nice to meet you. I'm here to help with any math problems you might have. What's on your mind? Do you have a specific problem you'd like me to solve, or would you like some help with a particular math concept?


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [6]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    if "####" in response:
        ans_part = response.split("####")[-1].strip()
        match = re.search(r"(-?\d+(?:\,\d+)?(?:\.\d+)?)", ans_part)
        if match:
            return match.group(1).replace(",", "")

    regex = r"(?:Answer:|The answer is)\s*\$?([0-9,.]+)"
    match = re.search(regex, response, re.IGNORECASE)
    if match:
        return match.group(1).replace(",", "")

    numbers = re.findall(r"(-?\d+(?:\,\d+)?(?:\.\d+)?)", response)
    return numbers[-1].replace(",", "") if numbers else None

In [7]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama-3.1-8b-instant",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []
    num_samples = min(num_samples, len(dataset))

    for i in tqdm(range(num_samples)):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        time.sleep(1)

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            # 5개마다 또는 마지막 샘플일 때 로그 출력
            if (i + 1) % 5 == 0 or (i + 1) == num_samples:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [8]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [ ]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Solve these. The answer format might vary. Sometimes use numbers, sometimes words.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        correct_answer = float(train_dataset['answer'][i].split("####")[-1].strip())
        
        wrong_answer = str(int(correct_answer * 0.5) if i % 2 == 0 else int(correct_answer * 1.5))

        prompt += f"\nQ{i+1}: {cur_question}\n"
        prompt += f"Result: {wrong_answer} (maybe)\n"

    prompt += "\nQ: {question}\nResult:"

    return prompt

In [10]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:14<00:21,  4.22s/it]

Progress: [5/10]
Current Acc.: [60.00%]


100%|██████████| 10/10 [00:22<00:00,  2.24s/it]

Progress: [10/10]
Current Acc.: [60.00%]


In [ ]:
shots = [0, 3, 5]

for shot in shots:
    print(f"\n>>> Running Direct Prompting: {shot}-shot")
    PROMPT = construct_direct_prompt(shot)
    
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        num_samples=50,
        VERBOSE=False
    )
    
    filename = f"direct_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)
    print(f"Saved results to {filename} with accuracy: {accuracy}")


>>> Running Direct Prompting: 0-shot


 10%|█         | 5/50 [00:07<01:04,  1.44s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:14<00:58,  1.47s/it]

Progress: [10/50]
Current Acc.: [70.00%]


...

Progress: [40/50]
Current Acc.: [75.00%]


 90%|█████████ | 45/50 [01:23<00:07,  1.57s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [01:31<00:00,  1.82s/it]

Progress: [50/50]
Current Acc.: [78.00%]
Saved results to direct_prompting_5.txt with accuracy: 0.78


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [ ]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    
    prompt = (
        "Solve the problem step by step. End with #### [value]\n"
    )

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset[idx]['question']
        cur_answer = train_dataset[idx]['answer']

        prompt += f"\nExample {i+1}:\n{cur_question}\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    
    return prompt

In [ ]:
import time

shots = [0, 3, 5]
total_start_time = time.time()

print("="*70)
print("CHAIN-OF-THOUGHT (CoT) PROMPTING BENCHMARK")
print("="*70)

for shot in shots:
    shot_start_time = time.time()
    print(f"\n{'='*70}")
    print(f"Testing CoT Prompting with {shot} shot(s)")
    print(f"{'='*70}")
    
    PROMPT = construct_CoT_prompt(shot)
    
    if shot == 0:
        PROMPT = "Question:\n{question}\nAnswer:"

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        num_samples=50,
        VERBOSE=False
    )
    
    filename = f"CoT_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)
    
    shot_elapsed = time.time() - shot_start_time
    print(f"\n✅ Completed: {shot}-shot")
    print(f"   📁 File: {filename}")
    print(f"   📊 Accuracy: {accuracy:.2%}")
    print(f"   ⏱️  Time: {shot_elapsed:.1f}s ({shot_elapsed/60:.1f} minutes)")

total_elapsed = time.time() - total_start_time
print(f"\n{'='*70}")
print(f"CHAIN-OF-THOUGHT (CoT) PROMPTING BENCHMARK COMPLETE")
print(f"Total Time: {total_elapsed:.1f}s ({total_elapsed/60:.1f} minutes)")
print(f"{'='*70}")

CHAIN-OF-THOUGHT (CoT) PROMPTING BENCHMARK

Testing CoT Prompting with 0 shot(s)


  0%|          | 0/50 [00:00<?, ?it/s]

 10%|█         | 5/50 [00:07<01:06,  1.47s/it]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:14<01:00,  1.51s/it]

...

Progress: [40/50]
Current Acc.: [80.00%]


 90%|█████████ | 45/50 [01:32<00:07,  1.49s/it]

Progress: [45/50]
Current Acc.: [80.00%]


100%|██████████| 50/50 [01:39<00:00,  1.99s/it]

Progress: [50/50]
Current Acc.: [82.00%]

✅ Completed: 5-shot
   📁 File: CoT_prompting_5.txt
   📊 Accuracy: 82.00%
   ⏱️  Time: 99.7s (1.7 minutes)

CHAIN-OF-THOUGHT (CoT) PROMPTING BENCHMARK COMPLETE
Total Time: 275.8s (4.6 minutes)


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [14]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(num_examples: int = 3):
    train_dataset = gsm8k_train
    sampled_indices = random.sample(range(len(train_dataset)), num_examples)

    prompt = (
        "Solve the math problem using the DUP method:\n"
        "Stage 1 [Core Question]: Extract the most detailed central goal.\n"
        "Stage 2 [Info]: List all necessary facts related to Stage 1.\n"
        "Stage 3 [Answer]: Solve step-by-step using Stage 1 & 2 info.\n"
        "Final result format: #### [value]\n"
    )

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset[idx]['question']
        cur_answer = train_dataset[idx]['answer']
        prompt += f"\nQ:{cur_question}\nA:{cur_answer}\n"
    
    prompt += "\nQ:{question}\nA:"

    return prompt

In [ ]:
import time

shots = [0, 3, 5]
total_start_time = time.time()

print("="*70)
print("MY PROMPTING BENCHMARK")
print("="*70)

for shot in shots:
    shot_start_time = time.time()
    print(f"\n{'='*70}")
    print(f"Testing My Prompting with {shot} shot(s)")
    print(f"{'='*70}")
    
    PROMPT = construct_my_prompt(shot)

    if shot == 0:
        PROMPT = PROMPT.format(question="{question}") + " Let's extract the Core Question and Key Info first, then solve."
    
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        num_samples=50,
        VERBOSE=False
    )
    
    filename = f"My_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)
    
    shot_elapsed = time.time() - shot_start_time
    print(f"\n✅ Completed: {shot}-shot")
    print(f"   📁 File: {filename}")
    print(f"   📊 Accuracy: {accuracy:.2%}")
    print(f"   ⏱️  Time: {shot_elapsed:.1f}s ({shot_elapsed/60:.1f} minutes)")

total_elapsed = time.time() - total_start_time
print(f"\n{'='*70}")
print(f"MY PROMPTING BENCHMARK COMPLETE")
print(f"Total Time: {total_elapsed:.1f}s ({total_elapsed/60:.1f} minutes)")
print(f"{'='*70}")

MY PROMPTING BENCHMARK

Testing My Prompting with 0 shot(s)


 10%|█         | 5/50 [00:08<01:14,  1.65s/it]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:16<01:05,  1.64s/it]

Progress: [10/50]
Current Acc.: [90.00%]


...

Progress: [40/50]
Current Acc.: [85.00%]


 90%|█████████ | 45/50 [01:12<00:07,  1.60s/it]

Progress: [45/50]
Current Acc.: [84.44%]


100%|██████████| 50/50 [01:20<00:00,  1.61s/it]

Progress: [50/50]
Current Acc.: [86.00%]

✅ Completed: 5-shot
   📁 File: My_prompting_5.txt
   📊 Accuracy: 86.00%
   ⏱️  Time: 80.6s (1.3 minutes)

MY PROMPTING BENCHMARK COMPLETE
Total Time: 251.2s (4.2 minutes)


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!